<center>
<h1> SOLUCIÓN TAREA 4</h1>
<h3>Diego Godoy Rojas & Luis Carlos Diaz </h3>
</center>


<h1>PRIMER PUNTO</h1> Implementar, en OpenCV, tres de los filtros para reducción de ruido Speckle vistos en clase.

<h1>SEGUNDO PUNTO</h1>Implementar, en OpenCV, cinco de las técnicas de estimación del nivel de ruido vistas en clase.

<h1>TERCER PUNTO</h1>Evaluar el desempeño de los filtros implementados (estimar el nivel de ruido en las imágenes antes y después de filtradas), al ser aplicados sobre imágenes reales y sobre imágenes sintéticas (creadas artificialmente), con
diferentes niveles de ruido multiplicativo.